In [ ]:
# MODULE 3: SCENE ASSESSMENT WITH LLaVA
# Version: v2.0 (SAGAI v1.1)
# Git repo: https://github.com/perezjoan/SAGAI
# This script performs batch visual scoring of street-level images using
# LLaVA-Next (v1.6 Mistral) loaded directly from Hugging Face. The processor
# handles image preprocessing, tokenization, and chat formatting, while the
# model generates a concise numeric response based on a task-specific prompt.
# Users define a structured prompt composed of four elements—role description,
# theory or scoring guide, task instruction, and required output format.
# The model then applies this rubric consistently to each image in the input
# folder. The script loops through all valid images stored in Google Drive,
# extracts the model’s numeric output, and saves results into a resumable CSV
# file.
#This makes it well suited for large-scale, zero-shot visual assessment
# tasks such as classification, counting, and spatial measurement.

In [ ]:
# ===============================================
# BLOCK 1 — Install dependencies + Hugging Face login
# ===============================================

!pip install --no-cache-dir -q transformers accelerate bitsandbytes

from google.colab import userdata
from huggingface_hub import login

# Load token stored in Colab Secrets (name: HF_TOKEN)
HF_TOKEN = userdata.get('HF_TOKEN')

if HF_TOKEN is None:
    raise ValueError("❌ HF_TOKEN not found in Colab secrets. Add it in 'Settings → Notebook secrets'")
else:
    print("✅ HuggingFace token loaded:", HF_TOKEN[:10] + "...")

login(token=HF_TOKEN)


In [ ]:
# ===============================================
# BLOCK 2 — Mount Google Drive + Define Paths
# ===============================================

from google.colab import drive
import os

# Mount Drive
drive.mount('/content/drive')

# User parameters
case_study = "nice"         # e.g., "vienna", "nice"
selected_task = "T1"        # Options: T1 = CATEGORIZATION, T2 = COUNTING, T3 = MEASURING
display_images = True # Display images for visual verification (optional)

# Folder structure
root_path = f"/content/drive/MyDrive/SAGAI"
image_folder = os.path.join(root_path, f"StreetViewBatchDownload_{case_study.capitalize()}")
output_path = os.path.join(root_path, f"Image_Analysis/Score_Analysis_LLaVA_{case_study.capitalize()}_{selected_task}.csv")

# Ensure output directory exists
os.makedirs(os.path.dirname(output_path), exist_ok=True)

print("📁 Image folder:", image_folder)
print("💾 Output file:", output_path)


In [ ]:
# ===============================================
# BLOCK 3 — Load LLaVA v1.6 Mistral (HuggingFace)
# ===============================================

from transformers import LlavaNextProcessor, LlavaNextForConditionalGeneration
import torch

# Model ID — change if needed
model_id = "llava-hf/llava-v1.6-mistral-7b-hf"

# Load processor (tokenizer + image processor + chat template)
processor = LlavaNextProcessor.from_pretrained(
    model_id,
    token=HF_TOKEN
)

# Load model (4-bit quantization to fit GPU memory)
model = LlavaNextForConditionalGeneration.from_pretrained(
    model_id,
    token=HF_TOKEN,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    load_in_4bit=True
).to("cuda")

print("🔥 LLaVA v1.6 Mistral loaded successfully!")


In [ ]:
# ===============================================
# BLOCK 4 — SAGAI Image Analysis Function (LLaVA-Next)
# ===============================================

from PIL import Image
from io import BytesIO
import requests
import re

def caption_image(
    image_file,
    prompt,
    do_sample=True,
    temperature=0.3,
    top_p=0.9,
    max_new_tokens=10
):
    """
    Sends an image + prompt to LLaVA-Next and returns ONLY the numeric score,
    based on the task prompt provided in Block 5.
    """

    # -------------------------
    # Load image (local or URL)
    # -------------------------
    if isinstance(image_file, str) and image_file.startswith("http"):
        image = Image.open(BytesIO(requests.get(image_file).content)).convert("RGB")
    else:
        image = Image.open(image_file).convert("RGB")

    # ----------------------------------------------------
    # Build conversation structure required by LLaVA-Next
    # ----------------------------------------------------
    conversation = [
        {
            "role": "user",
            "content": [
                {"type": "image"},                 # placeholder indicating an image is included
                {"type": "text", "text": prompt},  # the task prompt (T1/T2/T3)
            ],
        }
    ]

    # -------------------------------------------------------
    # Format the conversation into a ready-to-tokenize string
    # (using the model's built-in chat template)
    # -------------------------------------------------------
    prompt_string = processor.apply_chat_template(
        conversation,
        add_generation_prompt=True,  # instruct model to prepare for generation
        tokenize=False               # return a string, not tokens
    )

    # -----------------------------------------------------
    # Convert text + image into tensors expected by model
    # -----------------------------------------------------
    inputs = processor(
        images=image,
        text=prompt_string,
        return_tensors="pt"
    )

    # Move tensors to GPU
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    # ------------------------
    # Generate model response
    # ------------------------
    output = model.generate(
        **inputs,
        do_sample=do_sample,
        temperature=temperature,
        top_p=top_p,
        max_new_tokens=max_new_tokens
    )

    # -------------------------
    # Decode generated text
    # -------------------------
    raw = processor.decode(output[0], skip_special_tokens=True).strip()

    # ------------------------------------------------------------------
    # Remove the instruction prefix ([INST] ... [/INST]) if present
    # LLaVA-Next outputs may include the full prompt before the answer.
    # ------------------------------------------------------------------
    if "[/INST]" in raw:
        raw = raw.split("[/INST]")[-1].strip()

    # ----------------------------------------------------------
    # Extract ONLY numeric content (0/1/2 or width like "1.5")
    # ----------------------------------------------------------
    numbers = re.findall(r"-?\d+(?:\.\d+)?", raw)

    # If at least one number was found → return the last one
    if len(numbers) > 0:
        return numbers[-1]

    # If model output contains no numeric value → return NA
    return "NA"


In [ ]:
# ===============================================
# BLOCK 5 — Build task-specific prompt
# ===============================================

if selected_task == "T1":  # CATEGORIZATION
    role_description = (
        "You are an AI trained to visually analyze street-level images. "
        "Your task is to determine whether the environment shown is urban or rural."
    )
    theory_model = (
        "Classification Guide:\n"
        "- 0 = Rural area: sparse built environment or natural landscapes.\n"
        "- 1 = Urban area: dense built environment or visible infrastructure."
    )
    task = "Return ONLY 0 or 1."
    response_format = "Answer format: 0 or 1"

elif selected_task == "T2":  # COUNTING
    role_description = (
        "You are an AI trained to visually detect commercial storefronts."
    )
    theory_model = (
        "Scoring Guide:\n"
        "- 0 = No visible shops\n"
        "- 1 = One visible shop\n"
        "- 2 = More than one shop"
    )
    task = "Return ONLY 0, 1, or 2."
    response_format = "Answer format: 0, 1, or 2"

elif selected_task == "T3":  # MEASURING
    role_description = (
        "You are an AI trained to estimate sidewalk width from street-level images."
    )
    theory_model = (
        "Scoring Guide:\n"
        "- 0 = No visible sidewalk\n"
        "- Otherwise return sidewalk width rounded to nearest 0.5 meters."
    )
    task = "Return ONLY one number (0, 1.0, 1.5, 2.0, etc.)."
    response_format = "Answer format: 0 or a number like 1.5"

else:
    raise ValueError("Invalid selected_task: choose T1, T2, or T3")

# Final prompt
prompt = f"""
{role_description}

{theory_model}

{task}

{response_format}
"""

print("Prompt built for task:", selected_task)
print(prompt)

# ===============================================
# GLOBAL SAMPLING SETTINGS (used for ALL tasks)
# ===============================================
do_sample = True # Enable sampling (adds variation)
temperature = 0.3 # Low temperature for controlled creativity
top_p = 0.9 # Nucleus sampling within top 90% probable tokens
max_new_tokens = 10 # Expecting short numeric response


In [ ]:
# ===============================================
# BLOCK 6 — Run SAGAI Scoring on Image Folder
# ===============================================

import csv
import time
import pandas as pd
from IPython.display import display

# Start timer
start_time = time.time()

# ------------------ Load already processed images ------------------
already_processed = set()

if os.path.exists(output_path) and os.path.getsize(output_path) > 0:
    df_existing = pd.read_csv(output_path)
    already_processed = set(df_existing["image_name"].tolist())
    print(f"🔁 Resuming previous run — {len(already_processed)} images already scored.")
else:
    print("🆕 Starting fresh — no previous CSV found.")

# ------------------ Open CSV (append mode) ------------------
with open(output_path, "a", newline="") as csvfile:
    writer = csv.writer(csvfile)

    # Write header only if file is empty
    if os.path.getsize(output_path) == 0:
        writer.writerow(["image_name", "score"])

    # Iterate through images
    for fname in sorted(os.listdir(image_folder)):
        if not fname.lower().endswith((".jpg", ".jpeg", ".png")):
            continue

        if fname in already_processed:
            continue  # Skip already processed

        image_path = os.path.join(image_folder, fname)

        # Handle files marked as missing (from your GSV downloader)
        if "_NA" in fname:
            print(f"⚠️ Skipping NA image: {fname}")
            writer.writerow([fname, "NA"])
            continue

        print(f"\n🔍 Processing: {fname}")

        try:
            # Run inference
            result = caption_image(
                image_path,
                prompt,
                do_sample,
                temperature,
                top_p,
                max_new_tokens
            )

            print(f"📝 Score: {result}")

            # Optional display
            if display_images:
                display(Image.open(image_path))

            # Save result
            writer.writerow([fname, result.strip()])

        except Exception as e:
            print(f"❌ Error on {fname}: {e}")
            writer.writerow([fname, f"ERROR: {e}"])

# ------------------ Summary ------------------
elapsed_time = time.time() - start_time

print("\n✅ SAGAI scoring completed.")
print(f"📄 Results saved to: {output_path}")
print(f"⏱️ Total processing time: {elapsed_time:.2f} seconds")
